In [1]:
import os
import re
import pickle
import json
import pandas as pd
from pandas import Series,DataFrame

from utils.util import save_pickle, load_pickle

In [63]:
fp = './meta_data/event_definition/'

In [65]:
event_dict = {}
for fn in os.listdir(fp):
    if not fn.endswith('.txt'):
        continue
    print(fn)
    d = pd.read_csv(fp + fn, sep='?', header=None)
    l = d.to_dict(orient = 'split', index=False)['data']
    for i in l:
        key = i[0].strip()
        event_dict[key] = i[1:]
        event_dict[key].append(fn.split('.')[0])

BUSINESS.txt
CONFLICT.txt
CONTACT.txt
JUSTICE.txt
LIFE.txt
MOVEMENT.txt
PERSONNEL.txt
TRANSACTION.txt


In [67]:
save_pickle(event_dict, './event_definition.pkl')

In [69]:
def replace_span(j):
    jr = {}
    for event in j:
        jr[event] = {}
        for arg in j[event]:
            arg_re = arg.strip().replace('-Arg', '')
            jr[event][arg_re] = []
            name = ''
            for entity in j[event][arg][0].split('/'):
                try:
                    name = name + map_dict[entity] + '/'
                except:
                    print(event, j[event][arg])
                    continue
            jr[event][arg_re] = [j[event][arg][1], name[:-1]]
    return jr

In [71]:
fp = './meta_data/arg_roles/'

map_dict = {'PER':'Person',
            'ORG':'Organization',
            'GPE':'Geo-Political',
            'LOC':'Locations',
            'FAC':'Facilities',
            'VEH':'Vehicle',
            'WEA':'Weapon',
            'TIME':'Time',
            'CRIME':'Crime',
            'JOB':'Job',
            'MONEY':'Money',
            'NUM':'Number',
            'SEN':'Sentence'
           }

In [73]:
arg_role_definition = {}
for fn in os.listdir(fp):
    if not fn.endswith('.txt'):
        continue
    print('---' + fn)
    with open(fp + fn, 'r', encoding = 'utf-8') as f:
        s = f.read()
    j = json.loads(s)
    arg_role_definition.update(replace_span(j))

---Business.txt
---Conflict.txt
---Contact.txt
---Justice.txt
---Life.txt
---Movement.txt
---Personnel.txt
---Transaction.txt


In [75]:
practical_roles = load_pickle('./exp_data/practical_roles.pkl')

In [77]:
role_definition = {}
for event in arg_role_definition:
    role_definition[event] = {}
    role_list = arg_role_definition[event].keys()
    practical_list = practical_roles[event].keys()
    for role in role_list:
        if role in practical_list:
            role_definition[event][role] = arg_role_definition[event][role]
        else:
            print(f'{event} skip: {role}')

SENTENCE skip: Sentence
FINE skip: Money
EXTRADITE skip: Person
DIVORCE skip: Place


In [81]:
event_dict_full = {}
for i in event_dict:
    event_dict_full[i] = [event_dict[i][0], role_definition[i], event_dict[i][1], event_dict[i][2], event_dict[i][3]]

In [83]:
with open('./meta_data/event_dict_full.json', 'w', encoding='utf-8') as f:
    json.dump(event_dict_full, f)

In [85]:
with open('./meta_data/event_dict_full.json', 'r', encoding='utf-8') as f:
    event_dict_full = json.load(f)

In [21]:
len(event_dict_full)

33

In [23]:
print(event_dict_full['ELECT'])

['An ELECT Event occurs whenever a candidate wins an election designed to determine the PERSON argument of a START-POSITION Event. ', {'Person': ['the person elected', 'Person'], 'Entity': ['the voting agent(s)', 'Person/Organization/Geo-Political'], 'Place': ['Where the election takes place', 'Geo-Political/Locations/Facilities']}, 'Greg Lashutka was elected mayor of Columbus in 1993.', 'elected', 'PERSONNEL']


In [25]:
event_dict_full.keys()

dict_keys(['START-ORG', 'MERGE-ORG', 'DECLARE-BANKRUPTCY', 'END-ORG', 'ATTACK', 'DEMONSTRATE', 'MEET', 'PHONE-WRITE', 'ARREST-JAIL', 'RELEASE-PAROLE', 'TRIAL-HEARING', 'CHARGE-INDICT', 'SUE', 'CONVICT', 'SENTENCE', 'FINE', 'EXECUTE', 'EXTRADITE', 'ACQUIT', 'APPEAL', 'PARDON', 'BE-BORN', 'MARRY', 'DIVORCE', 'INJURE', 'DIE', 'TRANSPORT', 'START-POSITION', 'END-POSITION', 'NOMINATE', 'ELECT', 'TRANSFER-OWNERSHIP', 'TRANSFER-MONEY'])